In [ ]:
import torch
import torchvision
from torchvision.models.segmentation import segmentation
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import matplotlib.image as mpimg
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Set device to GPU_indx if GPU is avaliable
GPU_indx = 0
device = torch.device(GPU_indx if torch.cuda.is_available() else 'cpu')

In [ ]:
cd drive/MyDrive/AI3/

In [ ]:
net = segmentation.lraspp_mobilenet_v3_large(pretrained=False, progress=False, num_classes=3).to(device)
print(net)
Pretrained_Path = "./Models/bdd_city_320_w_mapil.pt"
check_point = torch.load(Pretrained_Path)
net.load_state_dict(check_point['model_state_dict'])

LRASPP(
  (backbone): IntermediateLayerGetter(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): Identity()
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1),

<All keys matched successfully>

In [ ]:
# To test on any jpg file

image = mpimg.imread('ped2.jpg')
resized_image = cv2.resize(image, (320,320), interpolation = cv2.INTER_AREA)

print(image.shape)

processed_img_t=np.moveaxis(resized_image,-1,0)
processed_img_t=torch.tensor(processed_img_t).unsqueeze(0)
out = net(processed_img_t.to(device).float())['out']
om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()

masked = np.ma.masked_where(om == 0, om)
plt.figure(figsize=(10,10))
plt.imshow(resized_image)
plt.imshow(masked,cmap = "jet_r", alpha=0.5)

print(np.unique(om))

In [ ]:
x = np.load('test.npz', mmap_mode='r')
image = x['arr_0']
label = x['arr_1']
print(image.shape)
print(label.shape)

In [ ]:
# Calculation of MIoU and mean inference time

x = np.load('test.npz', mmap_mode='r')
image = x['arr_0']
label = x['arr_1']

iou_data = np.array([])
time_data = np.array([])
for i in range(30):
    ground_image=image[i][:][:][:]
    ground_label=label[i][:][:][:]
    ground_label=np.squeeze(ground_label)

    processed_img_t=np.moveaxis(ground_image,-1,0)
    processed_img_t=torch.tensor(processed_img_t).unsqueeze(0)

    start_time = time.time()
    out = net(processed_img_t.to(device).float())['out']
    om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()

    time_data = np.append(time_data,(time.time() - start_time))



    outputs = om
    labels = ground_label
    intersection = (outputs & labels).sum((0, 1))
    union = (outputs | labels).sum((0, 1))

    iou = (intersection) / (union)
    iou_data = np.append(iou_data,iou)

print(time_data.mean())
print(iou_data.mean())

In [ ]:
# Code to show intersection and union of the output and the ground truth label

i=143
ground_image=image[i][:][:][:]
ground_label=label[i][:][:][:]

ground_label=np.squeeze(ground_label)

processed_img_t=np.moveaxis(ground_image,-1,0)
processed_img_t=torch.tensor(processed_img_t).unsqueeze(0)
out = net(processed_img_t.to(device).float())['out']
print(out.shape)
om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()

print(np.unique(om))
masked = np.ma.masked_where(om == 0, om)
plt.figure(figsize=(10,10))
plt.imshow(ground_image)
plt.imshow(masked,cmap = "jet_r", alpha=0.5)




#######
# To show IoU

# SMOOTH = 1e-6

ground_label_car = np.where((ground_label >= 2)&(ground_label < 3),1,0) 
# plt.figure(figsize=(10,10))
# plt.imshow(ground_label_car)

om_car = np.where((om >= 2)&(om < 3),1,0) 
# plt.figure(figsize=(10,10))
# plt.imshow(om_car)




ground_label_person = np.where((ground_label >= 1)&(ground_label < 2),1,0) 
# plt.figure(figsize=(10,10))
# plt.imshow(ground_label_person)

om_person = np.where((om >= 1)&(om < 2),1,0) 
# plt.figure(figsize=(10,10))
# plt.imshow(om_person)


outputs = om_car
labels = ground_label_car
intersection = (outputs & labels)
union = (outputs | labels)

# q=om_person
# a = np.ma.masked_where(q == 0, q)
# plt.figure(figsize=(10,10))
# plt.imshow(ground_image)
# plt.imshow(a, alpha=0.8)